https://dev.to/m_nevin/facial-analysis-with-python-and-amazon-rekognition-2il9

https://docs.aws.amazon.com/code-samples/latest/catalog/code-catalog-python-example_code-rekognition.html


https://docs.aws.amazon.com/rekognition/latest/dg/images-displaying-bounding-boxes.html

In [ ]:
import boto3

# Create a Rekognition client
client=boto3.client('rekognition','eu-west-2')

### Data on S3
The way to store blobs on AWS - to use with Rekognition we just specify the bucket and file, pass it to Rekognition through our client and get our response.

* Make sure you have your S3 Bucket, your accounts settings and rekognition in the same region

In [ ]:
response = client.detect_faces(
    Image={
         'S3Object':{
             'Bucket':'nsadawi-aws-s3',
             'Name':'49015677_2339570492995844_8862427911626424320_n.jpg'
          }
     },
    Attributes=['ALL']
)

### Local data

In [ ]:
with open("/Users/csstnns/Desktop/Yam.jpeg", 'rb') as image:
    response = client.detect_faces(Image={
        'Bytes': image.read()
     },
     Attributes=['ALL'])

In [ ]:
#response

In [ ]:
## Emotions
for faceDetail in response['FaceDetails']:
    print('Emotions: \t Confidence\n')
    for emotion in faceDetail['Emotions']:
        print(str(emotion['Type']) + '\t\t' + str(emotion['Confidence']))
        #print('\n')

In [ ]:
# Face Details
for faceDetail in response['FaceDetails']:
    print(faceDetail['Eyeglasses'])

### Draw Bounding Box Around Faces

In [ ]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor

def show_faces(photo,bucket): 
    
    client=boto3.client('rekognition', region_name='eu-west-2')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3', region_name='eu-west-2')
    s3_object = s3_connection.Object(bucket, photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectFaces 
    response = client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        Attributes=['ALL'])

    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
                    

    # calculate and display bounding boxes for each detected face       
    print('Detected faces for ' + photo)    
    for faceDetail in response['FaceDetails']:
        print('The detected face is between ' + str(faceDetail['AgeRange']['Low']) 
              + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
        
        box = faceDetail['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']
                

        print('Left: ' + '{0:.0f}'.format(left))
        print('Top: ' + '{0:.0f}'.format(top))
        print('Face Width: ' + "{0:.0f}".format(width))
        print('Face Height: ' + "{0:.0f}".format(height))

        points = (
            (left,top),
            (left + width, top),
            (left + width, top + height),
            (left , top + height),
            (left, top)

        )
        draw.line(points, fill='#00d400', width=2)

        # Alternatively can draw rectangle. However you can't set line width.
        #draw.rectangle([left,top, left + width, top + height], outline='#00d400') 

    image.show()

    return len(response['FaceDetails'])

def main():
    bucket="nsadawi-aws-s3"
    photo="49015677_2339570492995844_8862427911626424320_n.jpg"
    
    faces_count=show_faces(photo,bucket)
    print("faces detected: " + str(faces_count))


if __name__ == "__main__":
    main()

## Celebrity Detection

In [ ]:
with open("/Users/csstnns/Desktop/rm.png", 'rb') as image:
    response = client.recognize_celebrities(Image={
        'Bytes': image.read()
     })
    #print(response)
    # /Users/csstnns/Documents/viva3.jpg

## Scence Detection

In [ ]:
with open("/Users/csstnns/Desktop/rm.png", 'rb') as image:
    response = client.detect_labels(Image={
        'Bytes': image.read()
     })
    #print(response)

## Face Comparison

In [ ]:
import boto3

def compare_faces(sourceFile, targetFile):

    client=boto3.client('rekognition', region_name='eu-west-2')
   
    imageSource=open(sourceFile,'rb')
    imageTarget=open(targetFile,'rb')

    response=client.compare_faces(SimilarityThreshold=80,
                                  SourceImage={'Bytes': imageSource.read()},
                                  TargetImage={'Bytes': imageTarget.read()})
    
    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']
        similarity = str(faceMatch['Similarity'])
        print('The face at ' +
               str(position['Left']) + ' ' +
               str(position['Top']) +
               ' matches with ' + similarity + '% confidence')

    imageSource.close()
    imageTarget.close()     
    return len(response['FaceMatches'])          

def main():
    source_file='/Users/csstnns/Downloads/noureddin1.jpg'
    target_file='/Users/csstnns/Downloads/noureddin123.jpg'
    face_matches=compare_faces(source_file, target_file)
    print("Face matches: " + str(face_matches))


if __name__ == "__main__":
    main()

## Detection of Text in Image 

In [ ]:
import boto3

def detect_text(photo, bucket):

    client=boto3.client('rekognition', region_name='eu-west-2')

    response=client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
                        
    textDetections=response['TextDetections']
    print ('Detected text\n----------')
    for text in textDetections:
            print ('Detected text:' + text['DetectedText'])
            print ('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
            print ('Id: {}'.format(text['Id']))
            if 'ParentId' in text:
                print ('Parent Id: {}'.format(text['ParentId']))
            print ('Type:' + text['Type'])
            print()
    return len(textDetections)

In [ ]:
bucket='nsadawi-aws-s3'
photo='realmadrid.png'
text_count=detect_text(photo,bucket)
print("Text detected: " + str(text_count))